# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
!import pymorphy2

/bin/bash: import: command not found


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
from nltk.metrics.distance import (
edit_distance,
edit_distance_align,
binary_distance,
jaccard_distance,
masi_distance,
interval_distance,
custom_distance,
presence,
fractional_presence,
)

with open('litw-win.txt', encoding='cp1251') as f:
    words = [l.split()[1] for l in f]
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''
text_1 = ''
for i in list(text.split()):
    if i in words:
        text_1 += i
    else:
        min = 1000
        ind = 0
        for j in range(len(words)):
            a = edit_distance(i, words[j] )
            if a<=min:
                min = a
                ind = j
        text_1 += words[ind]
    text_1 += ' '

print(text_1)

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий 


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
text_2 = text.split()

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
import pandas as pd
import numpy as np 
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('preprocessed_descriptions.csv')
words = set()

for i in range(len(data)):
    w = []
    if type(data.preprocessed_descriptions[i])== str:
        w.append(word_tokenize(data.preprocessed_descriptions[i]))
    for j in range(len(w)):
        words.update(w[j])
words = list(words)
print(words[:5])

['sitting', '7mg', 'ducasse', 'stephgerlette', 'efforti']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
import random
for i in range(5):
    a = random.randint(0,len(words))
    b = random.randint(0,len(words))
    c = edit_distance(words[a], words[b])
    print(f'Между "{words[a]}" и "{words[b]}", {c} шагов')

Между "panel" и "cascarecce", 8 шагов
Между "forefront" и "highersodium", 10 шагов
Между "enjera" и "infinitum", 8 шагов
Между "responders" и "smokehouse", 9 шагов
Между "loxstyle" и "httpenlightenedcookingblogspotcom", 29 шагов


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def funk(word,k):
    global words
    k1 = 0
    s = []
    while True:
        for i in range(len(words)):
            a = edit_distance(word, words[i] )
            if a ==  k1:
                s.append(words[i])
            if len(s)>=k:
                break
        if len(s)>=k:
            break
        k1 +=1      
    return s
        

funk("sitting", 3)

['sitting', 'fitting', 'setting']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.stem import SnowballStemmer, WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

col1 = [i for i in range(len(words))]
sss = SnowballStemmer('english')
col2 = [sss.stem(word) for word in words]
lll = WordNetLemmatizer()
col3 = [lll.lemmatize(word) for word in words]
d = pd.DataFrame({'word': col1, 'stemmed_word':col2, 'normalized_word':col3}) 
d

,word,stemmed_word,normalized_word
0,0,sit,sitting
1,1,7mg,7mg
2,2,ducass,ducasse
3,3,stephgerlett,stephgerlette
4,4,efforti,efforti
...,...,...,...
32863,32863,carrott,carrott
32864,32864,reqular,reqular
32865,32865,108mg,108mg
32866,32866,canyou,canyou


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)
w = words
before = len(w)
for i in range(len(w)):
    if w[i] in stop_words:
        w.pop(i)
        print('ddd')
print(len(w), len(words))
print('Доля от общего кол-ва: ', (len(words)-len(w))/len(w))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
import csv
recipes = []
with open("preprocessed_descriptions.csv", "r") as file:
    csvFile = csv.reader(file)
    for row in csvFile:
        recipes += [row[1]]


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).